# This script gets the trial of subjective equality from a single's participant choice data.

0. Import the relevant packages

In [1]:
import pandas as pd
import os

# 1. Input the participant's session id and retrieve the parameters from the utility model

In [11]:
participant_code = input ("")
print('the Participant Code is ' + participant_code)


the Participant Code is 23_IDM_0021


In [12]:
#Define files & data pathway (Local Folder)
# participant_code = '23_IDM_0021'
split_dir = '/Volumes/UCDN/datasets/IDM/split/'
participant_trials = os.path.join(split_dir,'{}/crdm/{}_crdm.csv'.format(participant_code,participant_code))
print('loading fn: {}'.format(participant_trials))
trials = pd.read_csv(participant_trials)
parameters = pd.read_csv('csv/utility_CRDM_analysis.csv')

loading fn: /Volumes/UCDN/datasets/IDM/split/23_IDM_0021/crdm/23_IDM_0021_crdm.csv


In [13]:
#Define the parameter values for the given participant. This declares the varaibles we are gonna work with
# and gets the data from the CSV file
filtered_parameters = parameters[parameters['subject'] == participant_code]

alpha = filtered_parameters['alpha'].values[0]
beta = filtered_parameters['beta'].values[0]
gamma = filtered_parameters['gamma'].values[0]
print ("alpha: " + str(alpha))
print ("beta: " + str(beta))
print ("gamma: " + str(gamma))

alpha: 0.495323857
beta: 0.221680178
gamma: 4.100226383


# 2. The trial of subjective equality is that one with the lowest SV difference between the safe option and lottery option

![](Softmax_formula.png)

* We want to find the trial which has the closes p(choosing) to 0.50 (50%). That would be the trial that has the smallest SV difference betwwen the SVsafe and SValternative

In [14]:
#We create a new column in the SV difference CSV to know which trial has the lowest SV difference (Local folder)
utility_dir = '/Volumes/UCDN/datasets/IDM/utility'
participants_SV = os.path.join(utility_dir,"{}/crdm/{}_crdm_SV_hat.csv".format(participant_code,participant_code))
SV_delta = pd.read_csv(participants_SV)
num_trials = SV_delta.shape[0]
SV_delta['trial'] = range(1, num_trials + 1)
SV_delta.to_csv('SVdelta_trials', index = False)

In [15]:
'''We compute the absolute value and assign it to a trial'''
# Change1
# Read the CSV file into a DataFrame
data = pd.read_csv('SVdelta_trials')

# Specify the column you want to analyze
column_name = 'SV_delta'

# Calculate the absolute values
data['AbsoluteValue'] = data[column_name].abs()

# Find the value with the least absolute value
min_value = data[column_name].loc[data['AbsoluteValue'].idxmin()]

#Knowing the lowest SV delta value, we look for the trial
filtered_SV_delta = data[data['SV_delta'] == min_value]
SE = filtered_SV_delta['trial'].values[0]
print('The trial closest to Subjective Equalitiy is trial #' + str(SE))

The trial closest to Subjective Equalitiy is trial #28


In [16]:
#Now, we go to the trial CSV file to look up the structure of trial 28
filtered_trials = trials.loc[trials['crdm_trial_type'] == 'task'] #We should add the new column without filtering first.
num_trials = filtered_trials.shape[0]
filtered_trials['trial'] = range(1, num_trials + 1)


/var/folders/yx/fkwj2lbx3674djc_4jq5g8vnvntbk4/T/ipykernel_14964/4293071350.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_trials['trial'] = range(1, num_trials + 1)


In [17]:
filtered_trials = trials.copy()  # Make a copy of the DataFrame

filtered_trials.loc[filtered_trials['crdm_trial_type'] == 'task', 'trial'] = range(1, num_trials + 1)

filtered_trials.to_csv('csv/trials_num.csv', index=False)  # Save DataFrame to a CSV file



In [18]:
#Let's find the structure of the subjective equality trial
SE_trial = filtered_trials[filtered_trials['trial'] == SE]
crdm_amb_lev = SE_trial['crdm_amb_lev'].values[0]
crdm_sure_amt = SE_trial['crdm_sure_amt'].values[0]
crdm_lott = SE_trial['crdm_lott_top'].values[0] + SE_trial['crdm_lott_bot'].values[0]
prob = SE_trial ['crdm_lott_p'].values[0]

#Confirm the identities
print ('Certain $' + str(crdm_sure_amt))
print ("Lottery $" + str(crdm_lott))
print ("Probability of winning " + str(prob) + "%")
print ('Ambiguity level ' + str(crdm_amb_lev))

Certain $5.0
Lottery $20.0
Probability of winning 50.0%
Ambiguity level 0.0


In [19]:
#Format it to the desired input
cols = ['crdm_lott_amt', 'crdm_lott_p', 'crdm_amb_lev', 'alpha', 'beta', 'gamma']

df_indif = pd.DataFrame([[crdm_lott, prob, crdm_amb_lev, alpha, beta, gamma]], columns= cols)
print(df_indif)
subject_SE = 'csv/subject_input.csv'
df_indif.to_csv(subject_SE)

   crdm_lott_amt  crdm_lott_p  crdm_amb_lev     alpha     beta     gamma
0           20.0         50.0           0.0  0.495324  0.22168  4.100226
